# Forecasting: XGBoost

### Import Libraries and Load Data

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import shapiro
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings("ignore")

full_data = pd.read_csv("./data/goog.csv", usecols=[0, 1, 2, 3, 4, 5], parse_dates=True, index_col=0)

Check missing values

In [ ]:
full_data.info()

In [ ]:
full_data.isnull().sum().any()

In [ ]:
full_data.describe().T

### General Observations
1. The dataset contains 2989 observations.
2.  The attributes are all numeric except for a date column that is used as the index.
3.  There are no missing values in the dataset.
4.  The variable Close can be used as a target for the models.

Note: If we had categorical columns, we would need to identify them and encode them or extract a feature from it

## Find High Correlated Columns

Correlation measures the strength and direction of a linear relationship between two numerical variables. It ranges from -1 to 1:
- +1: perfect positive correlation (they move together)
- 0: no linear relationship
- -1: perfect negative correlation (they move in opposite directions)

Understanding correlation helps identify redundant features in a dataset. Removing highly correlated columns can simplify models, reduce overfitting, and improve performance.

In [ ]:
# get pairwise correlation matrix between numeric columns
corr = full_data.corr(numeric_only=True)

# get the absolute values, now our only interest is to know if they're highly correlated
cor_matrix = corr.abs()
upper_triangle_matrix = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(bool))

# set a correlation threshold
corr_th=0.90

# Create a list of columns to frop in case thye have a higher correlation than the thrshold
drop_list = [col for col in upper_triangle_matrix.columns if any(upper_triangle_matrix[col] > corr_th)]
drop_list

When analyzing feature correlation, we often remove highly correlated input variables to reduce redundancy and improve model performance. However, if a **column is your target variable** (the one you’re trying to predict), it **should never be dropped**, even if it’s highly correlated with other features. In fact, a strong correlation between features and the target is often a **good sign**, as it means the model can learn meaningful relationships. You should only drop **input features** that are highly correlated with each other — not with the target.

In [ ]:
# plot matrix heatmap
sns.set(rc={'figure.figsize': (10, 8)})
sns.heatmap(corr, cmap="RdBu", annot=True, fmt=".2f", vmin=-1, vmax=1)
plt.title(f"Matriz de correlación (umbral = {corr_th})")
plt.show()

Instead of keeping multiple highly correlated features like `High` and `Low`, we can take advantage of their relationship by engineering a **new, more informative feature**. 

For example, the difference between `High` and `Low` reflects the daily price range, which captures volatility in a single value. This reduces redundancy while preserving valuable information. By creating a feature like `price_range = High - Low`, we simplify the model input without losing predictive power.

In [ ]:
full_data["price_range"] = full_data["High"] - full_data["Low"]
full_data.head()

## Normality of Numerical Values

The **Shapiro-Wilk test** is a statistical test used to assess whether a numeric variable is **normally distributed**.
- Null hypothesis (H₀): the data **follows** a normal distribution.
- Alternative hypothesis (H₁): the data **does not** follow a normal distribution.

If the **p-value < 0.05**, we reject the null hypothesis — meaning the data is **not normally distributed**.

In [ ]:
print("\n📊 Shapiro-Wilk Normality Test Results📊")
print("==========================================\n")

num_cols = [col for col in full_data.columns if full_data[col].dtypes in ["int", "float"]]

for col in num_cols:
    stat, p = shapiro(full_data[col].dropna())  
    if p > 0.05:
        result = "✅ Likely Normally Distributed"
    else:
        result = "❌ Not Normally Distributed"

    print(f"📌 **{col}**\n   - p-value: {p:.5f} → {result}\n")

We can check this nornality in a visual way, as well, let's plot an what an example of a Normal Distribution would look like:

In [ ]:
# simulate normal values
data = np.random.normal(loc=0, scale=1, size=1000)

# plot
sns.histplot(data, kde=True)
plt.title("Example of a Normal Distribution")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

num_cols = [col for col in full_data.columns if full_data[col].dtypes in ["int", "float"]]

for col in num_cols:
    plt.figure()
    sns.histplot(full_data[col], kde=True)
    plt.title(f"Histogram of {col}")
    plt.show()

## Summmary of observations

### General Observations
- The dataset contains **2,989 rows** and **6 numerical columns**, plus one **date column**.
- There are **no missing values** in the dataset.
- All columns are numeric except for the **`Date`** column, which can be used later for time-based features.
- The **`Close`** column is a good candidate for the **target variable** in predictive models.

---

### Correlation Analysis

- We found that **`High`**, **`Low`**, and **`Close`** are **highly correlated** (correlation > 0.90).
- Since `Close` will be used as the **target**, it should **not be dropped**, even if it's highly correlated with other features.
- To reduce feature redundancy, we engineered a new variable:
  ```python
  full_data["price_range"] = full_data["High"] - full_data["Low"]

---

### Normality Testing
- We applied the **Shapiro-Wilk test to check** for normal distribution in numeric variables.
- All features returned **p-values < 0.0001**, meaning we **reject the null hypothesis** of normality.
- This indicates that the variables are **not normally distributed**, which is common in financial datasets due to **skewness** and **outliers**.
- However, since we plan to use tree-based models like XGBoost, **normality is not required** and does not impact model performance.

## Preparing Data for Training

Drop those columns we decided aren't going to add any value or that the model can't handle. 

Here we'll separate a 10% of the data for later testing and split de target variable as well.

In [ ]:
n = len(full_data)
test_size = int(n * 0.1)

train = full_data.iloc[:-test_size]
test = full_data.iloc[-test_size:]

columns_to_drop = ['High', 'Low']

X = train.copy()
X.drop(columns_to_drop, axis = 1, inplace = True)
y = X.pop('Close')

In [ ]:
X.shape, y.shape

Split into Train and Validation Split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
plt.figure(figsize=(10, 5))
sns.kdeplot(y_train, label='Train', fill=True)
sns.kdeplot(y_val, label='Validation', fill=True)
plt.title("Distribution of Target Variable in Train vs Validation Sets")
plt.legend()
plt.show()

Note: [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html)

If we had categorial columns or missing values in pur columns, we'd need to implement some preprocessing steps with sklearn's preprocessing capabilities, something like:

```python
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(transformers=[
    ('cat', categorical_transformer, new_cat_cols),
    ('num', numerical_transformer, new_num_cols)
])

## Define Machine Learning Models and Params to use

In [ ]:
# In case we want to test more models, here we can add them in the dicts
models = {
    'XGBoost': XGBRegressor(random_state=42, verbosity=0)
}

# we would also need to stablish parameters for those model here
param_grid = {
    'XGBoost': {
    'classifier__n_estimators': [100, 200],
    'classifier__learning_rate': [0.01, 0.1],
    'classifier__max_depth': [3, 5],
    'classifier__min_child_weight': [1, 3],
    'classifier__subsample': [0.8],
    'classifier__colsample_bytree': [0.8]
    }
}

## Hyperparameter Optimization

To optimize model performance, we will use [**Grid Search**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html), a technique that systematically explores a predefined set of hyperparameters. This method evaluates all possible combinations using cross-validation to find the configuration that yields the **best results** for the given task.

In [ ]:
# Keep results
best_models = {}
model_scores = []

for model_name, model in models.items():
    print(f"🔍 Training & Tuning {model_name}...")

    # Find best params
    grid_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid[model_name],
        n_iter=20,
        cv=3,
        scoring='r2',
        n_jobs=-1,
        random_state=42
    )
    grid_search.fit(X_train, y_train)

    # Keep best model
    best_models[model_name] = grid_search.best_estimator_

    # Predict
    y_pred = grid_search.best_estimator_.predict(X_val)

    # Evaluate
    r2 = r2_score(y_val, y_pred)
    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_val, y_pred)

    # Keep scores
    model_scores.append([model_name, r2, mse, rmse, mae])

    # Log summary
    print(f"✅ Finished training {model_name}")
    print(f"   R²: {r2:.4f} | MSE: {mse:.4f} | RMSE: {rmse:.4f} | MAE: {mae:.4f}")
    print("------------------------------------------------------")

In [ ]:
scores_df = pd.DataFrame(model_scores, columns=["Model", "R²", "MSE", "RMSE", "MAE"])
scores_df

In [ ]:
X_test = test.copy()
y_test = X_test.pop('Close')
X_test.drop(columns_to_drop, axis = 1, inplace = True)

In [ ]:
# test set prediction
fina_moldel = best_models['XGBoost']
y_test_pred = final_model.predict(X_test)

# Create df to compare values over time
results = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_test_pred
}, index=y_test.index)

# Plot
plt.figure(figsize=(14, 6))
plt.plot(results.index, results['Actual'], label='Actual', linewidth=2)
plt.plot(results.index, results['Predicted'], label='Predicted', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.title('📈 Predicted vs Actual Close Price Over Time (Test Set)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# make sure the index is ordered
train = train.sort_index()
test = test.sort_index()

# Add 5 day moving average
train['SMA_5'] = train['Close'].rolling(window=5).mean()
test['SMA_5'] = test['Close'].rolling(window=5).mean()

X_test = test.copy()
y_test = X_test.pop('Close')
X_test.drop(columns_to_drop, axis = 1, inplace = True)

In [ ]:
columns_to_drop = ['High', 'Low']

X_5ma = train.copy()
X_5ma.drop(columns_to_drop, axis = 1, inplace = True)
y_5ma = X_5ma.pop('Close')
X_train_5ma, X_val_5ma, y_train_5ma, y_val_5ma = train_test_split(X_5ma, y_5ma, test_size=0.2, random_state=42)

In [ ]:
def train_and_evaluate_models(X_train, X_val, y_train, y_val, models, param_grid, n_iter=20, cv=3, scoring='r2'):
    """
    Trains and evaluates multiple models using RandomizedSearchCV.
    
    Parameters
    ----------
    X_train, X_val, y_train, y_val : pd.DataFrame / pd.Series
        Training and validation data.
    models : dict
        Dictionary with model names and their instances.
    param_grid : dict
        Dictionary of hyperparameters for tuning.
    n_iter : int
        Number of hyperparameter combinations to try.
    cv : int
        Number of cross-validation folds.
    scoring : str
        Metric to optimize.
    
    Returns
    -------
    best_models : dict
        Trained models with the best parameters.
    model_scores : list
        Evaluation metrics for each model.
    """
    best_models = {}
    model_scores = []

    for model_name, model in models.items():
        print(f"🔍 Training & Tuning {model_name}...")

        grid_search = RandomizedSearchCV(
            estimator=model,
            param_distributions=param_grid[model_name],
            n_iter=n_iter,
            cv=cv,
            scoring=scoring,
            n_jobs=-1,
            random_state=42
        )
        grid_search.fit(X_train, y_train)

        best_model = grid_search.best_estimator_
        best_models[model_name] = best_model

        y_pred = best_model.predict(X_val)

        r2 = r2_score(y_val, y_pred)
        mse = mean_squared_error(y_val, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_val, y_pred)

        model_scores.append([model_name, r2, mse, rmse, mae])

        print(f"✅ Finished training {model_name}")
        print(f"   R²: {r2:.4f} | MSE: {mse:.4f} | RMSE: {rmse:.4f} | MAE: {mae:.4f}")
        print("------------------------------------------------------")

    return best_models, model_scores

In [ ]:
best_models_5ma, model_scores_5ma = train_and_evaluate_models(
    X_train_5ma, X_val_5ma, y_train_5ma, y_val_5ma,
    models=models,
    param_grid=param_grid
)

In [ ]:
def plot_actual_vs_predicted_over_time(model, X_test, y_test, title_suffix='Test Set'):
    """
    Makes predictions using the given model and plots actual vs predicted values over time.
    
    Parameters
    ----------
    model : Trained model with a predict method
    X_test : pd.DataFrame
        Input data for prediction
    y_test : pd.Series or pd.DataFrame
        Actual values, must have a temporal index (dates)
    title_suffix : str, optional
        Text to include in the plot title (default is 'Test Set')
    
    Returns
    -------
    results : pd.DataFrame
        DataFrame with 'Actual' and 'Predicted' columns, indexed like y_test.index
    """
    y_test_pred = model.predict(X_test)

    results = pd.DataFrame({
        'Actual': y_test,
        'Predicted': y_test_pred
    }, index=y_test.index)

    plt.figure(figsize=(14, 6))
    plt.plot(results.index, results['Actual'], label='Actual', linewidth=2)
    plt.plot(results.index, results['Predicted'], label='Predicted', linestyle='--')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.title(f'📈 Predicted vs Actual Close Price Over Time ({title_suffix})')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return results

In [ ]:
final_model = best_models_5ma['XGBoost']
results_df = plot_actual_vs_predicted_over_time(final_model, X_test, y_test)

---

### Exercise

There is significant room for improvement in the model results. To enhance your approach, try the following:
1. Implement a different train-test split that respects the temporal nature of the data (i.e., avoid random splits that break the time order).
2. Propose and engineer at least two new features that could help reduce the model errors and improve prediction accuracy.
  > 💡 **Hint:** Study how you can incorporate past context with a feature. Are there more financial indicators studied that we can use?

Your goal is to experiment with these changes and observe how they affect the model’s performance and predictions.

---